In [1]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

In [2]:
#Read the csv file from WeatherPy

weather_csv_file = "../WeatherPy/weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,0,Bismarck,1,US,1585387328,86,46.81,-100.78,39.00,5.82
1,1,Jamestown,90,US,1585387328,65,42.10,-79.24,48.00,5.82
2,2,Padang,99,ID,1585387328,73,-0.95,100.35,83.91,3.94
3,3,Pecos,1,US,1585387329,22,31.42,-103.49,57.20,18.34
4,4,Rikitea,89,PF,1585387329,79,-23.12,-134.97,78.21,8.19


In [5]:
# Configure gmaps with API key.
gmaps.configure(api_key=gkey)

# Convert Humidity to float, store, and handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lon"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# To narrow down the DataFrame in order to find ideal weather conditions:
ideal_weather_df = weather_df

# Drop any rows not containing the following three conditions:

# A maximum temperature lower than 80 degrees but higher than 65.
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 65)]

# Wind speed less than 10 mph.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
61,61,Copalillo,0,MX,1585387342,58,18.03,-99.12,69.01,2.26
157,157,Zihuatanejo,0,MX,1585387366,50,17.63,-101.55,77.94,4.25
183,183,Esna,0,EG,1585387372,34,25.29,32.55,78.80,9.17
250,250,Jalu,0,LY,1585387206,22,29.03,21.55,69.42,6.15
257,257,Teocelo,0,MX,1585387387,32,19.38,-96.97,68.70,4.45
340,340,Murgab,0,TM,1585387404,20,37.50,61.97,66.20,6.93
482,482,Viedma,0,AR,1585387434,47,-40.81,-63.00,66.42,6.22
649,649,Corowa,0,AU,1585387473,48,-36.03,146.38,71.60,3.36
779,779,Barra de Tecoanapa,0,MX,1585387265,77,16.52,-98.75,73.36,3.53
821,821,Iralaya,0,HN,1585387511,85,15.00,-83.23,75.56,6.22


In [14]:
#Store the above narrowed down DataFrame into a new one

first_hotel_df = ideal_weather_df

# Add params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}


for index, row in first_hotel_df.iterrows():
    
    # Obtain Lat. and Long. from DataFrame
    lat = row["Lat"]
    lon = row["Lon"]

    # Change location after each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Assemble url and make API call
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
          
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        first_hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 61: Copalillo.
Missing field/result... skipping.
------------
Retrieving Results for Index 157: Zihuatanejo.
Closest hotel is La Casa Que Canta.
------------
Retrieving Results for Index 183: Esna.
Closest hotel is Orient Star Dahabyia Isna 05 Nights Each Sunday.
------------
Retrieving Results for Index 250: Jalu.
Closest hotel is Jalu Hotel.
------------
Retrieving Results for Index 257: Teocelo.
Closest hotel is Posada La Querencia.
------------
Retrieving Results for Index 340: Murgab.
Missing field/result... skipping.
------------
Retrieving Results for Index 482: Viedma.
Closest hotel is Hotel Cristal.
------------
Retrieving Results for Index 649: Corowa.
Closest hotel is Statesman Motor Inn.
------------
Retrieving Results for Index 779: Barra de Tecoanapa.
Closest hotel is Hotel Manglares.
------------
Retrieving Results for Index 821: Iralaya.
Closest hotel is Iglesia morava renovada.
------------
Retrieving Results for Index 909: Calvinia.
Closes

In [17]:

first_hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
61,61,Copalillo,0,MX,1585387342,58,18.03,-99.12,69.01,2.26,NaN
157,157,Zihuatanejo,0,MX,1585387366,50,17.63,-101.55,77.94,4.25,La Casa Que Canta
183,183,Esna,0,EG,1585387372,34,25.29,32.55,78.80,9.17,Orient Star Dahabyia Isna 05 Nights Each Sunday
250,250,Jalu,0,LY,1585387206,22,29.03,21.55,69.42,6.15,Jalu Hotel
257,257,Teocelo,0,MX,1585387387,32,19.38,-96.97,68.70,4.45,Posada La Querencia
340,340,Murgab,0,TM,1585387404,20,37.50,61.97,66.20,6.93,NaN
482,482,Viedma,0,AR,1585387434,47,-40.81,-63.00,66.42,6.22,Hotel Cristal
649,649,Corowa,0,AU,1585387473,48,-36.03,146.38,71.60,3.36,Statesman Motor Inn
779,779,Barra de Tecoanapa,0,MX,1585387265,77,16.52,-98.75,73.36,3.53,Hotel Manglares
821,821,Iralaya,0,HN,1585387511,85,15.00,-83.23,75.56,6.22,Iglesia morava renovada


In [20]:
#To create map:

# Assign the marker layer to a variable

locations = first_hotel_df[["Lat", "Lon"]]
markers = gmaps.marker_layer(locations)


# Using this template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row

first_hotel_info = [info_box_template.format(**row) for index, row in first_hotel_df.iterrows()]


# Create hotel symbol layer

first_hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=first_hotel_info
)

# Add the layer to the map

fig.add_layer(markers)
fig.add_layer(first_hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))